In [28]:
from datastructures.graphutils import draw_graph_graphical_props, flat_graph_formulation, Node, VAR, COMP, SOLVER, merge_edges, sources
from datastructures.graphutils import nested_sources, all_variables, edges_to_Ein_Eout, namefromid
from datastructures.nestedgraph import build_typedgraph
from compute import Var
from datastructures.api import Model, adda, addf, addsolver, setsolvefor, addobj, addineq
from datastructures.execution import edges_from_components, generate_components_and_residuals, Component
from datastructures.operators import sort_scc, reorder_merge_solve
from datastructures.nestedgraph import build_typedgraph
from datastructures.workflow import get_f, implicit_comp_name, SOLVE, OPT, EQ, NEQ, OBJ, IMPL, EXPL, order_from_tree, ENDCOMP, default_solver_options
from datastructures.workflow import order_from_tree, default_solver_options, mdao_workflow
from datastructures.workflow_mdao import mdao_workflow_with_args
from datastructures.assembly import architecture_mappings, buildidpvars, build_archi, root_sources
from datastructures.runpipeline import nestedform_to_mdao
from datastructures.postprocess import print_outputs

In [2]:
from datastructures.nestedgraph import flat_graph_formulation
from datastructures.graphutils import root_solver

In [3]:
import openmdao.api as om

In [16]:
model = Model()
m = model.root
x = Var('x')
y = Var('y')
a = adda(m, 'a', x+2+y)
#a = Var('a', 1)
adda(m, x, 3*a-10);

In [17]:
edges = edges_from_components(model.components)
tree = model.Ftree, model.Stree, model.Vtree

In [18]:
edges

({0: ('x', 'y'), 1: ('a',)}, {0: ('a',), 1: ('x',)}, {})

In [19]:
G = flat_graph_formulation(*edges)
#merge_order = sort_scc(G)
#merge_parent = root_solver(tree) # all merged components will have this solver as the parent
#ordered_edges, ordered_tree = reorder_merge_solve(edges, tree, merge_order, merge_parent, True)

In [20]:
ordered_edges = edges
ordered_tree = tree

In [21]:
comp_options = model.comp_options
var_options = model.var_options
solvers_options = model.solvers_options

In [22]:
wf

[(SOLVE, 1, None, {'designvars': ()}, {}), (EXPL, 0, 1), (EXPL, 1, 1)]

In [23]:
nodetyperepr = {VAR: '{}', COMP: 'f_{}', SOLVER: 's_{}'}
namingfunc = namefromid(nodetyperepr)
sequence = order_from_tree(ordered_tree[0], ordered_tree[1], ordered_edges[1])
solvers_options = default_solver_options(ordered_tree, solvers_options)
wf = mdao_workflow(sequence, solvers_options, comp_options, var_options)
components = model.components
lookup_f = get_f(components, ordered_edges)
wfmdao = mdao_workflow_with_args(wf, lookup_f, namingfunc)

In [24]:
from datastructures.nestedgraph import sources, edges_to_Ein_Eout

In [25]:
prob, mdao_in, groups = build_archi(ordered_edges, ordered_tree, wfmdao, namingfunc, model.idmapping)

In [26]:
prob.run_model();


===
s_1
===
NL: Newton Converged in 1 iterations


In [29]:
print_outputs(model, prob, namingfunc)

$$a$$,3.889,
$$x$$,1.667,
